## Day 8: Treetop Tree House

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day08/puzzle.ipynb)


In [25]:
#!value --name sampleRaw 

30373
25512
65332
33549
35390

In [26]:
#load "../common/common.fsx"

type Map = int[,]


let parse s : Map = 
    Pattern1.read id s
    |> Array.map (Array.ofSeq >> Array.map (string >> int))
    |> array2D

In [27]:
#!share sampleRaw --from value

#load "../common/common.fsx"
#load "./formatting.fsx"

let heightToColor height = 
    System.Drawing.Color.FromArgb(int(((float height) * 25.6 * 0.9 + 25.6)), 0, 255, 0)

let sampleTrees = parse sampleRaw
sampleTrees
|> toDisplayable heightToColor
|> withSettings { Width = 200 }

In [28]:
#!value --name actualRaw --from-file ./data_actual.txt

In [29]:
#!share actualRaw --from value

let actualTrees = parse actualRaw
actualTrees
|> toDisplayable heightToColor
|> withSettings { Width = 99*5 }

### Part 1

In [30]:
module Part1 = 
    type Verdict = 
    | VisibleFromEdges
    | NotVisibleFromEdges

In [31]:
module Part1 = 
    let verdict i j height (trees:int[,]) = 
        let allSmaller = Seq.forall (fun h -> h < height)
        let visible1 = trees[..(i-1), j] |> allSmaller
        let visible2 = trees[(i+1).., j] |> allSmaller
        let visible3 = trees[i, ..(j-1)] |> allSmaller
        let visible4 = trees[i, (j+1)..] |> allSmaller
        if (visible1 || visible2 || visible3 || visible4)
        then Part1.VisibleFromEdges
        else Part1.NotVisibleFromEdges
    let verdicts trees =
        trees
        |> Array2D.mapi (
            fun i j height ->
                verdict i j height trees
        )
    let verdictToColor verdict = 
        match verdict with
        | Part1.VisibleFromEdges -> System.Drawing.Color.FromArgb(128, 255, 0, 255)
        | Part1.NotVisibleFromEdges -> System.Drawing.Color.Transparent


In [32]:
let samplePart1Result =
    Part1.verdicts sampleTrees

samplePart1Result
|> Array2D.toSeq 
|> Seq.countBy id
|> display

sampleTrees 
|> toDisplayable heightToColor
|> withMatrix samplePart1Result Part1.verdictToColor
|> withSettings { Width = 200 }

index,Item1,Item2
0,VisibleFromEdges,21
1,NotVisibleFromEdges,4


In [33]:
let actualPart1Result = 
    Part1.verdicts actualTrees

actualPart1Result
|> Array2D.toSeq
|> Seq.countBy id
|> display

actualTrees 
|> toDisplayable heightToColor
|> withMatrix actualPart1Result Part1.verdictToColor
|> withSettings { Width = 99*5 }

index,Item1,Item2
0,VisibleFromEdges,1763
1,NotVisibleFromEdges,8038


### Part 2

In [34]:
module Part2 = 
    let score i j height (trees:int[,]) = 

        let countSmaller = 
            Seq.takeUntil (fun h -> h >= height)
            >> Seq.length
        let score1 = trees[..(i-1), j] |> Seq.rev |> countSmaller
        let score2 = trees[(i+1).., j] |> countSmaller
        let score3 = trees[i, ..(j-1)] |> Seq.rev |> countSmaller
        let score4 = trees[i, (j+1)..] |> countSmaller
        score1 * score2 * score3 * score4

    let scores (trees:int[,]) =
        trees
        |> Array2D.mapi (
            fun i j height ->
                score i j height trees
        )



In [37]:
let samplePart2Result = 
    sampleTrees 
    |> Part2.scores
    |> Array2D.indexed
    |> Array2D.toSeq 
    |> Seq.maxBy (fun ((i,j),x) -> x)

display samplePart2Result

let toDisplayableScores (scores : int[,]) = 
    let aFrom = 10.0
    let aTo = 255.0
    let quantiles = [0.0 .. 1.0/(aTo - aFrom) .. 1.0]
    let allScoreValues =
        scores
        |> Array2D.toSeq
        |> Seq.sort
        |> Seq.distinct
        |> Seq.toArray
    let scoreMax = allScoreValues |> Array.last
    let scoreMap =
        allScoreValues
        |> Seq.indexed
        |> Seq.map (fun (i,score) -> (score,i))
        |> Map.ofSeq
    let count = scoreMap |> Map.count
    let scoreToColor score = 
        if (score = scoreMax) then System.Drawing.Color.Fuchsia
        else
            let a = scoreMap[score]
            System.Drawing.Color.FromArgb(float a / float count * (aTo - aFrom) + aFrom |> int, 0, 255, 0)
    scores |> toDisplayable scoreToColor

sampleTrees 
|> Part2.scores
|> toDisplayableScores
|> withSettings { Width = 200 }

Item1,Item2
"( 3, 2 )",8


In [38]:
let actualPart2Result = 
    actualTrees 
    |> Part2.scores
    |> Array2D.indexed
    |> Array2D.toSeq 
    |> Seq.maxBy (fun ((i,j),x) -> x)

display actualPart2Result

actualTrees 
|> Part2.scores
|> toDisplayableScores
|> withSettings { Width = 99*5 }

Item1,Item2
"( 47, 78 )",671160
